# Numerical exercise QM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
h_bar = 1
m = 1
k_0 = 20
L = 20

N = 1000

omega = h_bar * k_0**2 / (2*m) # Obs. is this correct??
E = h_bar * omega # Is this correct??

dx = L/(N-1)

In [ ]:
x = np.linspace(0, L, N) # Spacing should be almost dx



In [ ]:
def timestep(psi_im, psi_re, v, dt):
    """Performs one timestep of size dt
    Input:
        psi_im : Psi_im(x, t)
        psi_re : Psi_re(x, t + dt/2)
    Returns:
        psi_im : Psi_im(x, t + dt)
        psi_re : Psi_re(x, t + dt)
    """
    
    assert psi_im[0] == psi_im[-1] == psi_re[0] == psi_re[-1] == 0, "Psi should be zero for x = 0 and x = L"
    # Find psi_im(t+dt) given psi_im(t) and psi_re(t + dt/2)
    psi_im[1:-1] -= dt*(v[1:-1]*psi_re[1:-1]/h_bar - (psi_re[2:] - 2*psi_re[1:-1] + psi_re[:-2])*h_bar/(2*m*dx**2))
    
    # Find psi_re(t+ 3*dt/2) given psi_re(t+dt/2) and psi_im(t+dt) (remember, psi_im is now at t = t+dt)
    psi_re[1:-1] += dt*(v[1:-1]*psi_im[1:-1]/h_bar - (psi_im[2:] - 2*psi_im[1:-1] + psi_im[:-2])*h_bar/(2*m*dx**2))
    
    return psi_im, psi_re

In [ ]:
# Problem 1
x_s = 5
sigma_x = 1.5

# Normalization
C = 1/np.sqrt(
    np.sum(np.exp(-(x-x_s)**2/(sigma_x**2)))*dx
)

v = np.zeros(N)

dt = 0.1 * h_bar / (h_bar**2 /(2*m*dx**2) + np.max(v)) # Much smaller than ..., see (12)

num_iter = 3000
psi_re = np.zeros([num_iter, N])
psi_im = np.zeros([num_iter, N])
# see (8)
psi_re[0, 1:-1] = C*np.exp(-(x[1:-1]-x_s)**2/(2*sigma_x**2)) * np.cos(k_0*x[1:-1] - omega*dt/2)
psi_im[0, 1:-1] = C*np.exp(-(x[1:-1]-x_s)**2/(2*sigma_x**2)) * np.sin(k_0*x[1:-1])

In [ ]:
for i in range(num_iter-1):
    psi_im[i+1], psi_re[i+1] = timestep(psi_im[i], psi_re[i], v, 0.0001)
    if not i % 100 and False:
        plt.xlim([0, L])
        plt.plot(x, psi_im[i])
        plt.plot(x, psi_re[i])
        plt.show()

In [ ]:
# Plot start and final state
plt.plot(x, psi_re[0], label="Initial")
plt.plot(x, psi_re[-1], label="FInal")
plt.legend()
simulated_time = dt*num_iter
print(f"Simulated time: {simulated_time}s")

In [ ]:
from matplotlib import animation
from IPython.display import HTML
plt.rcParams.update({'animation.html':'html5', 'savefig.dpi': 100})

def init_anim():
    global ax, line1, line2
    line1, = ax.plot([], [])
    line2, = ax.plot([], [])
    ax.set_xlim([0, L])
    ax.set_ylim([-1, 1])
    
def animate(y):
    global ax, line
    line1.set_data(x, y[0])
    line2.set_data(x, y[1])
    
fig, ax = plt.subplots()
anim = animation.FuncAnimation(fig, animate, init_func=init_anim, frames=zip(psi_re[::100], psi_im[::100]), interval=60)
plt.close(anim._fig)
HTML(anim.to_html5_video())

## Introducing a barrier

In [ ]:
l = L/50 # barrier width
v_0 =  3*E/4 # barrier height
v = v_0*np.heaviside(x*np.heaviside(-x +L/2 +l/2, 1) -L/2+l/2, 0)

# Problem 3
x_s = 5
sigma_x = 1.5

In [ ]:
def simulate(v, x_s, sigma_x, num_iter):
    # Normalization
    C = 1/np.sqrt(
        np.sum(np.exp(-(x-x_s)**2/(sigma_x**2)))*dx
    )

    dt = 1 * h_bar / (h_bar**2 /(2*m*dx**2) + np.max(v)) # Much smaller than ..., see (12)
    psi_re = np.zeros([num_iter, N])
    psi_im = np.zeros([num_iter, N])
    # see (8)
    psi_re[0, 1:-1] = C*np.exp(-(x[1:-1]-x_s)**2/(2*sigma_x**2)) * np.cos(k_0*x[1:-1] - omega*dt/2)
    psi_im[0, 1:-1] = C*np.exp(-(x[1:-1]-x_s)**2/(2*sigma_x**2)) * np.sin(k_0*x[1:-1])

    for i in range(num_iter-1):
        psi_im[i+1], psi_re[i+1] = timestep(psi_im[i], psi_re[i], v, 0.0001) #todo: pass more sensible dt
        
    return psi_re, psi_im

In [ ]:
psi_re, psi_im = simulate(v, x_s, sigma_x, 6000)

In [ ]:
# Plot start and final state
plt.plot(x, psi_re[0], label="Initial")
plt.plot(x, psi_re[-1], label="FInal")
plt.legend()
simulated_time = dt*num_iter
print(f"Simulated time: {simulated_time}s")

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, v)
anim = animation.FuncAnimation(fig, animate, init_func=init_anim, frames=zip(psi_re[::100], psi_im[::100]), interval=60)
plt.close(anim._fig)
HTML(anim.to_html5_video())

In [ ]:
# Finding P(right) and P(left), probability of being right and left of the barrier
mid_i = int(N/2) # Index of center

P_right = np.sum((psi_re**2+psi_im**2)[[0, -1], mid_i:], axis=1)*dx
P_left = np.sum((psi_re**2+psi_im**2)[[0, -1], :mid_i], axis=1)*dx

print(f"P_right: {P_right}\nP_left: {P_left}\nSum: {P_right + P_left}")

In [ ]:
print(f"Probability of transmission: {P_right[1]}")

In [ ]:
# Find transmission probability as a function of barrier height
v = E*np.heaviside(x*np.heaviside(-x +L/2 +l/2, 1) -L/2+l/2, 0)
fraction = np.linspace(0, 3/2, 50)
P_trans = []

for f in fraction:
    psi_re, psi_im = simulate(f*v, x_s, sigma_x, 6000)
    P_trans.append(np.sum((psi_re**2+psi_im**2)[-1, mid_i:])*dx)

In [ ]:
plt.plot(fraction, P_trans)